In [3]:
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [ ]:
out_df = pd.read_json(f"../data/annotated/mfc_annotated_Mistral.json", lines=True)
len(out_df)
frame_map = json.load(open("../data/raw/mfc_v4.0/codes.json",'r'))

In [ ]:
clean_preds = []
for ind, pred in enumerate(out_df['predictions']):
    pred = pred.replace("\n","")
    reasoning_begins = [m.start()-1 for m in re.finditer('"reasoning"', pred)]
    reasoning_ends = [pred[loc:].find("}") for loc in reasoning_begins]
    frame_annos = []
    prev = 0
    try:
        for i, (b,e) in enumerate(zip(reasoning_begins, reasoning_ends)):
            if i == 0:
                try:
                    frame_json = json.loads(pred[1:b-1]+"}")
                    frame_json['reasoning'] = pred[b+15:b+e-1]
                    frame_annos.append(frame_json)
                except:
                    continue
            else:
                try:
                    frame_json = json.loads("{" + pred[prev+1:b-1] + "}")
                    frame_json['reasoning'] = pred[b+15:b+e-1]
                    frame_annos.append(frame_json)
                except:
                    continue
            if i < len(reasoning_begins)-1:
                gap = pred[b+e:].index("{")
                prev = b+e+gap
    except:
        pass
    clean_preds.append(frame_annos)

In [ ]:
out_df['clean_preds'] = clean_preds
out_df['pred_frames'] = out_df['clean_preds'].apply(lambda x: [frame_map[str(frame_json['frame_id'])+'.0'] if str(frame_json['frame_id'])+'.0' in frame_map.keys() else None for frame_json in x ])
good, bad = 0, 0
rank = []
correct = []
for ind, row in out_df[['pred_frames','label']].iterrows():
    if row['label'].strip() in row['pred_frames']:
        good += 1
        rank.append(row['pred_frames'].index(row['label'].strip()))
        correct.append(True)
    else:
        bad += 1
        correct.append(False)

In [ ]:
out_df['correct'] = correct
out_df[out_df['correct'] == 0].sample(100)[['text', 'label', 'pred_frames']]

In [ ]:
np.array(rank).mean()

In [ ]:
print(accuracy_score(labels, frame_names), f1_score(labels, frame_names, average='macro'))
cm = confusion_matrix(labels, frame_names)
cmd = ConfusionMatrixDisplay(cm, display_labels=list(out_df.label.unique()))
cmd.plot()
plt.show()